# install

In [ ]:
! git clone https://github.com/SuphanutN/Thai-NER-BiLSTMCRF-WordCharEmbedding

In [ ]:
!pip install datasets

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

# import

In [ ]:
# %tensorflow_version 1.x

In [ ]:
import keras
import tensorflow as tf
print(tf.__version__)

In [ ]:
DATA_PATH='/content/Thai-NER-BiLSTMCRF-WordCharEmbedding/'
RAW_PATH = f'{DATA_PATH}raw/'
MODEL_PATH = f'{DATA_PATH}model/Keras/WordCharModel/'
W_MODEL_PATH = f'{DATA_PATH}model/thai2fit/'
Dict_MODEL_PATH = f'{DATA_PATH}model/dictionary/'

In [ ]:
# Save / Load File
import dill
import pickle

# Plot Graph
import matplotlib.pyplot as plt

# Sklearn Report
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from itertools import chain

# Load Vectors
from gensim.models import KeyedVectors

# Utility
import numpy as np
import time

# Model Utility
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd

# Keras Model
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from tensorflow.keras.utils import to_categorical
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint
# from tensorflow import _keras_module

In [ ]:
!pip install tensorflow-addons

In [ ]:
# # Save / Load File
# import dill
# import pickle

# # Plot Graph
# import matplotlib.pyplot as plt

# # Sklearn Report
# import sklearn
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.preprocessing import LabelBinarizer
# from itertools import chain

# # Load Vectors
# from gensim.models import KeyedVectors

# # Utility
# import numpy as np
# import time

# # Model Utility
# from sklearn.model_selection import train_test_split
# from tqdm import tqdm
# import pandas as pd

# # Keras Model
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import load_model
# from tensorflow.keras import Model, Input
# from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
# from tensorflow.keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D
# from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.utils import to_categorical
# from tensorflow_addons.layers import CRF
# # from tensorflow.keras_contrib.layers import CRF
# # from tensorflow import _keras_module

# load data

In [ ]:
! wget https://github.com/kobkrit/datasets/raw/main/AIFORTHAI-LST20Corpus.tar.gz

In [ ]:
! tar -xvzf AIFORTHAI-LST20Corpus.tar.gz

In [ ]:
from datasets import load_dataset
lst20 = load_dataset("lst20", data_dir="/content/LST20_Corpus")

In [ ]:
import pandas as pd
train_df = pd.DataFrame(lst20['train'])
validation_df = pd.DataFrame(lst20['validation'])
test_df = pd.DataFrame(lst20['test'])

In [ ]:
train_df.head()

In [ ]:
tagged_sents = []
ner_kaggle=['B_BRN', 'B_DES', 'B_DTM', 'B_LOC', 'B_MEA', 'B_NUM', 'B_ORG', 'B_PER', 'B_TRM', 'B_TTL', 'E_BRN', 'E_DES', 'E_DTM', 'E_LOC', 'E_MEA', 'E_NUM', 'E_ORG', 'E_PER', 'E_TRM', 'E_TTL', 'I_BRN', 'I_DES', 'I_DTM', 'I_LOC', 'I_MEA', 'I_NUM', 'I_ORG', 'I_PER', 'I_TRM', 'I_TTL', 'O']

# train_df
for i in range(len(train_df['tokens'])):
  sent=[]
  for j in range(len(train_df['tokens'][i])):
    sent.append((train_df['tokens'][i][j], ner_kaggle[train_df['ner_tags'][i][j]]))
  tagged_sents.append(sent)

# validation_df
for i in range(len(validation_df['tokens'])):
  sent=[]
  for j in range(len(validation_df['tokens'][i])):
    sent.append((validation_df['tokens'][i][j], ner_kaggle[validation_df['ner_tags'][i][j]]))
  tagged_sents.append(sent)

In [ ]:
len(tagged_sents)

In [ ]:
a = 0
for t in tagged_sents:
  if len(t) > 150:
    a += 1
a

In [ ]:
tagged_sents = list(filter(lambda x: len(x) <= 150, tagged_sents))

In [ ]:
len(tagged_sents)

In [ ]:
# with open(RAW_PATH + 'datatrain.data', 'rb') as file:
#     datatofile = dill.load(file)
    
# tagged_sents = []
# for i in datatofile:
#     text_inside = []
#     for j in i:
#         text_inside.append((j[0],j[2]))
#     tagged_sents.append(text_inside)

train_sents, test_sents= train_test_split(tagged_sents, test_size=0.2, random_state=112)
print(len(train_sents))
print(len(test_sents))

# train_sents[0], test_sents[0], 
# len(train_sents[0]), len(test_sents[0])
train_sents[0]

# preprocessing

In [ ]:
thai2fit_model = KeyedVectors.load_word2vec_format(W_MODEL_PATH+'thai2vecNoSym.bin',binary=True)
thai2fit_weight = thai2fit_model.vectors

In [ ]:
thai2fit_weight.shape, thai2fit_weight[0]

In [ ]:
word_list=[]
ner_list=[]
thai2dict = {}

for sent in train_sents:
    for word in sent:
        word_list.append(word[0])
        ner_list.append(word[1])    
for word in thai2fit_model.index2word:
    thai2dict[word] = thai2fit_model[word]

word_list.append("pad")
word_list.append("unknown") #Special Token for Unknown words ("UNK")
ner_list.append("pad")
ner_kaggle.append('pad')

all_words = list(word_list)
all_ner = list(ner_list)
all_thai2dict = list(thai2dict)

word_to_ix = dict((c, i) for i, c in enumerate(all_words)) #convert word to index 
ner_to_ix = dict((c, i) for i, c in enumerate(ner_kaggle)) #convert ner to index
thai2dict_to_ix = dict((c, i) for i, c in enumerate(thai2dict)) #convert thai2fit to index 

ix_to_word = dict((v,k) for k,v in word_to_ix.items()) #convert index to word
ix_to_ner = dict((v,k) for k,v in ner_to_ix.items())  #convert index to ner
ix_to_thai2dict = dict((v,k) for k,v in thai2dict_to_ix.items())  #convert index to thai2fit

n_word = len(word_to_ix)
n_tag = len(ner_to_ix)
n_thai2dict = len(thai2dict_to_ix)
print(n_word)
print(n_tag)
print(n_thai2dict)
print(ner_to_ix)

In [ ]:
chars = set([w_i for w in thai2dict for w_i in w])
char2idx = {c: i + 5 for i, c in enumerate(chars)}

char2idx["pad"] = 0
char2idx["unknown"] = 1
char2idx[" "] = 2

char2idx["$"] = 3
char2idx["#"] = 4
char2idx["!"] = 5
char2idx["%"] = 6
char2idx["&"] = 7
char2idx["*"] = 8
char2idx["+"] = 9
char2idx[","] = 10
char2idx["-"] = 11
char2idx["."] = 12
char2idx["/"] = 13
char2idx[":"] = 14
char2idx[";"] = 15
char2idx["?"] = 16
char2idx["@"] = 17
char2idx["^"] = 18
char2idx["_"] = 19
char2idx["`"] = 20
char2idx["="] = 21
char2idx["|"] = 22
char2idx["~"] = 23
char2idx["'"] = 24
char2idx['"'] = 25

char2idx["("] = 26
char2idx[")"] = 27
char2idx["{"] = 28
char2idx["}"] = 29
char2idx["<"] = 30
char2idx[">"] = 31
char2idx["["] = 32
char2idx["]"] = 33

n_chars = len(char2idx)
print(n_chars)

In [ ]:
len(thai2dict)

# build model

In [ ]:
with open(Dict_MODEL_PATH+'chardict.pickle', 'wb') as chardict:
    pickle.dump(char2idx, chardict)
with open(Dict_MODEL_PATH+'nerdict.pickle', 'wb') as nerdict:
    pickle.dump(ner_to_ix, nerdict)

In [ ]:
max_len = 150
max_len_char = 30

character_LSTM_unit = 32
char_embedding_dim = 32
main_lstm_unit = 256 ## Bidirectional 256 + 256 = 512
lstm_recurrent_dropout = 0.5

train_batch_size = 32
train_epochs = 50

In [ ]:
def prepare_sequence_word(input_text):
    idxs = list()
    for word in input_text:
        if word in thai2dict:
            idxs.append(thai2dict_to_ix[word])
        else:
            idxs.append(thai2dict_to_ix["unknown"]) #Use UNK tag for unknown word
    return idxs

def prepare_sequence_target(input_label):
    idxs = [ner_to_ix[w] for w in input_label]
    return idxs

In [ ]:
input_sent =[ [ word[0] for word in sent]for sent in train_sents ] #words only
train_targets =[ [ word[1] for word in sent]for sent in train_sents ] #NER only

input_test_sent =[ [ word[0] for word in sent]for sent in test_sents ] #words only
test_targets =[ [ word[1] for word in sent]for sent in test_sents ] #NER only

In [ ]:
a = 0
ni = 1
for i in input_test_sent:
 if a < len(i):
    a = len(i)
    ni += 1
    print(ni, a)

In [ ]:
len(input_sent[0])

In [ ]:
ni_ = 0
nj_ = 0
for i,j in enumerate(input_sent):
  for jj in j:
    if len(j) > nj_:
      nj_ = len(j)
      ni_ = i
    else:
      pass
ni_ , nj_

In [ ]:
len(input_sent[5153])

In [ ]:
for i,j in enumerate(input_sent[5153]):
  print(i,j)

In [ ]:
## Word Training
X_word_tr = [prepare_sequence_word(s) for s in input_sent]
X_word_tr = pad_sequences(maxlen=max_len, sequences=X_word_tr, value=thai2dict_to_ix["pad"], padding='post', truncating='post')

In [ ]:
len(X_word_tr), len(X_word_tr[0])

In [ ]:
## Word Training
X_word_tr = [prepare_sequence_word(s) for s in input_sent]
X_word_tr = pad_sequences(maxlen=max_len, sequences=X_word_tr, value=thai2dict_to_ix["pad"], padding='post', truncating='post')

## Character Training
X_char_tr = []
for sentence in train_sents:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                if(sentence[i][0][j] in char2idx):
                    word_seq.append(char2idx.get(sentence[i][0][j]))
                else:
                    word_seq.append(char2idx.get("unknown"))
            except:
                word_seq.append(char2idx.get("pad"))
        sent_seq.append(word_seq)
    X_char_tr.append(np.array(sent_seq))

## Sequence Label Training
y_tr = [prepare_sequence_target(s) for s in train_targets]
y_tr = pad_sequences(maxlen=max_len, sequences=y_tr, value=ner_to_ix["pad"], padding='post', truncating='post')
# y_tr = pad_sequences(maxlen=n_tag, sequences=y_tr, value=ner_to_ix["pad"], padding='post', truncating='post')
y_tr = [to_categorical(i, num_classes=n_tag) for i in y_tr]

In [ ]:
## Word Testing
X_word_te = [prepare_sequence_word(s) for s in input_test_sent]
X_word_te = pad_sequences(maxlen=max_len, sequences=X_word_te, value=thai2dict_to_ix["pad"], padding='post', truncating='post')

## Character Testing
X_char_te = []
for sentence in test_sents:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                if(sentence[i][0][j] in char2idx):
                    word_seq.append(char2idx.get(sentence[i][0][j]))
                else:
                    word_seq.append(char2idx.get("unknown"))
            except:
                word_seq.append(char2idx.get("pad"))    
        sent_seq.append(word_seq)
    X_char_te.append(np.array(sent_seq))

## Sequence Label Testing
y_te = [prepare_sequence_target(s) for s in test_targets]
y_te = pad_sequences(maxlen=max_len, sequences=y_te, value=ner_to_ix["pad"], padding='post', truncating='post')
y_te = [to_categorical(i, num_classes=n_tag) for i in y_te]

## model

In [ ]:
# Word Input
word_in = Input(shape=(max_len,), name='word_input_')

# Word Embedding Using Thai2Fit
word_embeddings = Embedding(
                            input_dim=n_thai2dict,
                            output_dim=400,
                            weights = [thai2fit_weight],
                            input_length=max_len,
                            mask_zero=False,
                            name='word_embedding', 
                            trainable=False)(word_in)

# Character Input
char_in = Input(shape=(max_len, max_len_char,), name='char_input')

# Character Embedding
emb_char = TimeDistributed(Embedding(
                                              input_dim=n_chars, 
                                              output_dim=char_embedding_dim, 
                                              input_length=max_len_char, 
                                              mask_zero=False))(char_in)

# Character Sequence to Vector via BiLSTM
char_enc = TimeDistributed(Bidirectional(LSTM(
                                                              units=character_LSTM_unit, 
                                                              return_sequences=False, 
                                                              recurrent_dropout=lstm_recurrent_dropout)))(emb_char)


# Concatenate All Embedding
all_word_embeddings = concatenate([word_embeddings, char_enc])
all_word_embeddings = SpatialDropout1D(0.3)(all_word_embeddings)

# Main Model BiLSTM
main_lstm = Bidirectional(
      LSTM(
        units=main_lstm_unit, 
        return_sequences=True,
        recurrent_dropout=lstm_recurrent_dropout)
    )(all_word_embeddings)
    
main_lstm = TimeDistributed(Dense(50, activation="relu"))(main_lstm)

# CRF
crf = CRF(n_tag)  # CRF layer
out = crf(main_lstm)  # output

# Model
model = Model([word_in, char_in], out)

model.compile(
    optimizer="adam", 
    loss=crf.loss_function, 
    metrics=[crf.accuracy]
)

model.summary()

In [ ]:
# print(X_word_te.shape,np.array(X_char_te).reshape((len(X_char_te) , max_len, max_len_char)).shape)

In [ ]:
len(X_word_tr)

In [ ]:
filepath='/content/'+"weights-improvement-{epoch:02d}-{val_crf_viterbi_accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_crf_viterbi_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(
                    [
                      np.array(X_word_tr),
                      np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))
                    ],
                    np.array(y_tr),
                    batch_size=train_batch_size, 
                    epochs=train_epochs, 
                    verbose=1,
                    callbacks=callbacks_list,
                    validation_data=(
                        [
                         np.array(X_word_te), 
                         np.array(X_char_te).reshape((len(X_char_te) , max_len, max_len_char))
                         ],
                         np.array(y_te)
                    )
                )

model.save('/content/model.h5')

In [ ]:
hist = pd.DataFrame(history.history)

plt.style.use("ggplot")
plt.figure(figsize=(12,12))
plt.plot(hist["crf_viterbi_accuracy"])
plt.plot(hist["val_crf_viterbi_accuracy"])
plt.show()

In [ ]:
hist = pd.DataFrame(history.history)

plt.style.use("ggplot")
plt.figure(figsize=(12,12))
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])
plt.show()

In [ ]:
save_filepath=MODEL_PATH+"last_weight-50.hdf5"
model.save_weights(save_filepath)

In [ ]:
load_filepath=MODEL_PATH+"weights-improvement-46-0.996.hdf5"
model.load_weights(load_filepath)

In [ ]:
pred_model = model.predict([X_word_te,np.array(X_char_te).reshape((len(X_char_te),max_len, max_len_char))], verbose=1)


In [ ]:
y_pred = []
y_true = []

for i in range(0,len(pred_model)):
    try:
        out = np.argmax(pred_model[i], axis=-1)
        true = np.argmax(y_te[i], axis=-1)
        revert_pred=[ix_to_ner[i] for i in out]
        revert_true=[ix_to_ner[i] for i in true]
        y_pred.append(revert_pred)
        y_true.append(revert_true)
    except:
        print (i)
def ner_classification_report(y_true, y_pred):
 
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
    tagset = list(sorted(set(lb.classes_)))
    tagset = tagset[:-2]
    print(tagset)
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
        digits=4
    )
print(ner_classification_report(y_true,y_pred))